In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from FoodSeg103_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create scribbles

Load the ground truths as batch

In [2]:
img_nums = [1792] #[n for n in range(0, 4900, 500)] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

Total images: 1


Check and filter for resolution

In [3]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 250_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
1792:   196,608 pixels


In [4]:
# Filter out those images
gts = {im_num: gt for im_num, gt in gts.items() if resol[im_num] <= thresh}
num_imgs = len(gts)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 250,000: 1
1792: 196,608 pixels


Define scribbles parameters

In [5]:
# Which scribbles to create
bins = [0.05]#[0.05, 0.3, 1.5]
suff = ["w3"] #standing for width 3
sq_scaling = 100
scribble_width = 3
scribbles_seed = 1

# Where to save the scribbles
folder_path = "./FoodSeg103_results/data/run02"

save_res = False
show_res = True
print_steps = True

Loop and create scribbles

In [9]:
percentages = {}
for bin in bins:
    percentages[bin] = []
    for img_num, gt in gts.items():
        for s in suff:
            np.random.seed(scribbles_seed)
            print(f"IMG {img_num}: bin {bin}, suff {s}")
            scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, sq_scaling=sq_scaling, mode="all",
                                                            save_res=save_res, suff=s, show_res=show_res, image=None, print_steps=print_steps, scribble_width=scribble_width)
            percentages[bin].append(perc_labelled)
            print(f"Annotation percentage: {perc_labelled:2.2f}%\n")

IMG 1792: bin 0.05, suff w3
CLASS 1:
sk_max_pix: 12.33, sq_size: 44, sk_pix_range: (12, 88)
   prim_sk_squares pix: 12 = 0.02%
Adjusting square size and range to 22 (11, 88)
Adjusting square size and range to 11 (5, 88)
   sec_sk_squares pix: 11 = 0.01%
lines_max_pix: 12.33, line_pix_range: (12, 88)
--- Sampling lines: pix_in_sk 1382 indx_step 1 dist_to_edge 2
---    pix_in_line 7 added_pix 0
---    outside line_pix_range (12, 88)
---    pix_in_line 12 added_pix 0
---    pix_in_line 38 added_pix 12
---    overshoot nr. 0
---    pix_in_line 12 added_pix 12
---    overshoot nr. 1
---    pix_in_line 30 added_pix 12
---    overshoot nr. 2
---    pix_in_line 5 added_pix 12
---    overshoot nr. 3
---    pix_in_line 32 added_pix 12
---    overshoot nr. 4
---    pix_in_line 6 added_pix 12
---    overshoot nr. 5
---    pix_in_line 49 added_pix 12
---    overshoot nr. 6
---    pix_in_line 27 added_pix 12
---    overshoot nr. 7
---    pix_in_line 4 added_pix 12
---    overshoot nr. 8
---    pix_i

KeyboardInterrupt: 

In [ ]:
np.sum(scribbles>0) / np.prod(scribbles.shape)

Quick report of labelling percentages:

In [ ]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

Show the last created scribble with its ground truth, and all ground truths (optionally)

In [ ]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")

In [ ]:
for g in gts:
    v.add_labels(gts[g], name=f"GT {g}")